ControlNet
=====

**Adding Conditional Control to Text-to-Image Diffusion Models**

 * Paper: https://arxiv.org/pdf/2302.05543

![ControlNet Overview](../assets/controlnet_overview.png)

![ControlNet Overview](../assets/controlnet_arch.png)

```bash
git clone https://github.com/lllyasviel/ControlNet.git ControlNet_repo

sed -i 's,name: control$,name: controlnet,g' ControlNet_repo/environment.yaml

# upgrade transformers to be able to download 'openai/clip-vit-large-patch14'
pip install --upgrade transformers
```

 * Download the model using huggingface hub

```python
import shutil
from huggingface_hub import hf_hub_download
import os

repo_id = "lllyasviel/ControlNet"
filenames = [
    "models/control_sd15_canny.pth",
    "models/control_sd15_depth.pth",
    "models/control_sd15_seg.pth"
]

save_dir = "./models"

os.makedirs(save_dir, exist_ok=True)

for filename in filenames:
    cached_path = hf_hub_download(repo_id=repo_id, filename=filename)
    
    # Copy to your custom directory
    dst_path = os.path.join(save_dir, os.path.basename(filename))
    shutil.copyfile(cached_path, dst_path)
    print(f"Copied to: {dst_path}")

```

In [ ]:
import sys
sys.path.append("ControlNet_repo")

import cv2
import einops
import gradio as gr
import numpy as np
import torch
import random
from pytorch_lightning import seed_everything

from annotator.util import resize_image, HWC3
#from annotator.canny import CannyDetector
from cldm.model import create_model, load_state_dict
from cldm.ddim_hacked import DDIMSampler

from share import *
import config


/home/pyml/anaconda3/envs/controlnet/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


logging improved.


In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = create_model('ControlNet_repo/models/cldm_v15.yaml').cpu()
#model.load_state_dict(load_state_dict('./models/control_sd15_canny.pth', location='cuda'))
state_dict = load_state_dict('./models/control_sd15_canny.pth', location=device)
missing_keys, unexpected_keys = model.load_state_dict(state_dict, strict=False)
print(f"Missing keys: {missing_keys}")
print(f"Unexpected keys: {unexpected_keys}")
model = model.to(device)
ddim_sampler = DDIMSampler(model)

ControlLDM: Running in eps-prediction mode
DiffusionWrapper has 859.52 M params.
making attention of type 'vanilla' with 512 in_channels
Working with z of shape (1, 4, 32, 32) = 4096 dimensions.
making attention of type 'vanilla' with 512 in_channels
Loaded model config from [ControlNet_repo/models/cldm_v15.yaml]
Loaded state_dict from [./models/control_sd15_canny.pth]
Missing keys: []
Unexpected keys: ['cond_stage_model.transformer.text_model.embeddings.position_ids']
